Oulu_NLPTM_TwitterBrexit

In [12]:
import re
from collections import Counter
import nltk

In [2]:
def load(filename):
    tweet_list = []
    with open(filename, 'r') as f:
        for tweet in f:
            tweet_list.append(tweet.strip())
            
    print(f"loaded tweets from {filename}")
    return tweet_list

In [3]:
conservative_tweets = load("conservative_tweets_preprocessed.txt")
labour_tweets = load("labour_tweets_preprocessed.txt")

loaded tweets from conservative_tweets_preprocessed.txt
loaded tweets from labour_tweets_preprocessed.txt


In [4]:
#getting top 10 frequent words
def top10(tweets):
    list_of_words = []
    for tweet in tweets:
        words = tweet.split()
        list_of_words.append(words)
    flat_list = [item for sublist in list_of_words for item in sublist]

    word_counts = Counter(flat_list)
    return word_counts.most_common(10)

In [5]:
top10_conservative = top10(conservative_tweets)
top10_labour = top10(labour_tweets)
print('TOP10 words by conservatives: ')
for i, word in enumerate(top10_conservative):
    print(str(i+1) + ". wordstem:'" +  str(word[0]) + "' occurences: " + str(word[1]))

print()
print('TOP10 words by labour: ')
for i, word in enumerate(top10_labour):
    print(str(i+1) + ". wordstem:'" +  str(word[0]) + "' occurences: " + str(word[1]))


TOP10 words by conservatives: 
1. wordstem:'thank' occurences: 1128
2. wordstem:'work' occurences: 1052
3. wordstem:'today' occurences: 1045
4. wordstem:'peopl' occurences: 1003
5. wordstem:'support' occurences: 981
6. wordstem:'great' occurences: 943
7. wordstem:'get' occurences: 857
8. wordstem:'help' occurences: 673
9. wordstem:'busi' occurences: 617
10. wordstem:'local' occurences: 602

TOP10 words by labour: 
1. wordstem:'support' occurences: 1059
2. wordstem:'govern' occurences: 988
3. wordstem:'thank' occurences: 936
4. wordstem:'peopl' occurences: 849
5. wordstem:'need' occurences: 801
6. wordstem:'work' occurences: 675
7. wordstem:'pleas' occurences: 643
8. wordstem:'today' occurences: 586
9. wordstem:'get' occurences: 564
10. wordstem:'test' occurences: 524


In [15]:
top10_conservative_wordset = set([word[0] for word in top10_conservative])
top10_labour_wordset = set([word[0] for word in top10_labour])

jaccard_index_top10 = nltk.jaccard_distance(top10_conservative_wordset, top10_labour_wordset)

print('Jaccard distance based on TOP10 most frequent words: ' + str(jaccard_index_top10))

{'peopl', 'thank', 'busi', 'great', 'today', 'local', 'support', 'get', 'help', 'work'}
{'peopl', 'govern', 'thank', 'need', 'today', 'pleas', 'support', 'test', 'get', 'work'}
Jaccard distance based on TOP10 most frequent words: 0.5714285714285714
